In [ ]:
# Tile image 

In [ ]:
#| default_exp tile_image

In [ ]:
#| export
import torch
import torch.nn.functional as F

In [ ]:
a = torch.randn(1, 256, 3072, 3072)#.cuda()
b = torch.randn(256, 256, 3, 3)#.cuda()
c = torch.randn(256)#.cuda()

In [ ]:
#| export
# function to  process a tile
def process_tile(tile, kernel, bias):
    # apply convolution to the tile
    return F.conv2d(tile, kernel, bias, (1, 1), (1,1), (1,1),1)

In [ ]:
y_full = torch.zeros(1, 256, 3072, 3072)#.cuda()

In [ ]:
tile_size = 512
overlap = b.size(2) - 1 # kernel size - 1

In [ ]:
for i in range(0, a.shape[2], tile_size):
    for j in range(0, a.shape[3], tile_size):
        # get the tile
        start_i = max(i -overlap // 2, 0)
        end_i = min(i + tile_size + overlap // 2, a.shape[2])

        start_j = max(j - overlap // 2, 0)
        end_j = min(j + tile_size + overlap // 2, a.shape[3])

        # extract the tile

        tile = a[:, :, start_i:end_i, start_j:end_j]

        conv_tile = process_tile(tile, b, c)
        # get the tile's output
        y_tile = process_tile(tile, b, c)
        # add the tile's output to the full output

        output_start_i = i
        output_end_i = i + tile_size if i + tile_size <= a.shape[2] else a.shape[2]
        output_start_j = j
        output_end_j = j + tile_size if j + tile_size <= a.shape[3] else a.shape[3]


        # adjust the slicing of the convolved tile to match the output size
        tile_i_start = 0 if i ==0 else overlap // 2
        tile_i_end = conv_tile.shape[2] - (0 if i + tile_size >= a.shape[2] else overlap // 2)

        tile_j_start = 0 if j == 0 else overlap // 2
        tile_j_end = conv_tile.shape[3] - (0 if j + tile_size >= a.shape[3] else overlap // 2)

        y_full[:, :, output_start_i:output_end_i, output_start_j:output_end_j] = conv_tile[:, :, tile_i_start:tile_i_end, tile_j_start:tile_j_end]



        y_full[:, :, i:i+tile_size, j:j+tile_size] += y_tile[:, :, :tile_size, :tile_size]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)